In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import confusion_matrix, classification_report
from imblearn.under_sampling import RandomUnderSampler

AttributeError: module 'tensorflow.core.function.trace_type' has no attribute 'Serializable'

In [1]:
import tensorflow as tf

AttributeError: module 'tensorflow.core.function.trace_type' has no attribute 'Serializable'

In [5]:
data = pd.read_csv('../data/energy_task_new2.csv')
data.drop(['Unnamed: 0'], axis=1, inplace=True)
y = data['Appliances']
X = data.drop(['Appliances'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [3]:
data2 = pd.read_csv('../data/neo_task_new2.csv')
data2.drop(['Unnamed: 0'], axis=1, inplace=True)
num_0 = len(data2[data2['hazardous']==0])
num_1 = len(data2[data2['hazardous']==1])
undersampled_data2 = pd.concat([ data2[data2['hazardous']==0].sample(num_1) , data2[data2['hazardous']==1] ])
y_2 = undersampled_data2['hazardous']
X_2 = undersampled_data2.drop(['hazardous'], axis=1)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_2, y_2, stratify=y_2, test_size=0.2)

In [4]:
data2 = pd.read_csv('../data/neo_task_new2.csv')
data2.drop(['Unnamed: 0'], axis=1, inplace=True)
data2

,id,est_diameter_min,est_diameter_max,relative_velocity,miss_distance,absolute_magnitude,hazardous
0,3561024.0,0.016016,0.035813,56014.078517,1.024333e+06,26.10,0
1,54016766.0,0.030518,0.068240,7864.348060,3.268186e+07,24.70,0
2,3746620.0,0.055533,0.124177,55257.544508,6.538636e+07,23.40,0
3,3633054.0,0.019256,0.043057,41531.404722,1.260796e+07,25.70,0
4,3742124.0,0.139494,0.311918,67639.394481,7.130590e+07,21.40,0
...,...,...,...,...,...,...,...
90831,54231436.0,0.017561,0.039268,23264.740825,1.635007e+06,25.90,0
90832,3824972.0,0.110804,0.247765,24802.519406,3.351901e+07,21.90,0
90833,3740101.0,0.035039,0.078350,116288.999548,5.471396e+07,24.40,0
90834,3691093.0,0.044112,0.098637,45763.317060,2.694877e+07,23.90,0


In [5]:
num_0 = len(data2[data2['hazardous']==0])
num_1 = len(data2[data2['hazardous']==1])
print(num_0,num_1)
# случайный андерсэмплинг
undersampled_data = pd.concat([ data2[data2['hazardous']==0].sample(num_1) , data2[data2['hazardous']==1] ])
print(len(undersampled_data))

81996 8840
17680


In [6]:
y_classification = undersampled_data['hazardous']
X_classification = undersampled_data.drop(['hazardous'], axis=1)

In [7]:
X_classification_train, X_classification_test, y_classification_train, y_classification_test = train_test_split(X_classification,
                                                                                                                y_classification,
                                                                                                                stratify=y_classification,
                                                                                                                test_size=0.2)

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_classification_train)
X_classification_train = scaler.transform(X_classification_train)
X_classification_train

array([[ 2.37158928, -0.33416365, -0.33292289, -1.14898987, -0.83094134,
         0.08000733],
       [-0.46832142,  0.18399592,  0.18722351, -0.28519735,  1.53823009,
        -0.73901522],
       [-0.46000565,  0.3096296 ,  0.31333893,  2.05709329,  0.39219818,
        -0.85247289],
       ...,
       [-0.39728186, -0.5125681 , -0.51201143, -0.27132041, -1.55990643,
         0.89548432],
       [-0.46168841,  0.01836893,  0.02096144, -0.62048966, -0.76418024,
        -0.55819206],
       [-0.43259997,  0.56309376,  0.56777497,  1.48908409,  1.63774795,
        -1.04038715]])

In [9]:
X_classification_test = scaler.transform(X_classification_test)
X_classification_test

array([[-0.48357308,  0.8935207 ,  0.89946892, -0.39686383, -0.52017692,
        -1.23184696],
       [-0.46609854, -0.08448275, -0.08228461,  1.98519845,  0.22483414,
        -0.41991552],
       [-0.46022925, -0.24724857, -0.24567454, -1.17698304,  0.10387276,
         0.51492355],
       ...,
       [-0.42520451, -0.45718286, -0.45641381, -1.40958284, -1.47187765,
         0.54092911],
       [-0.39194225, -0.58347524, -0.58319045, -1.03941003, -1.71128198,
         1.95914997],
       [-0.38419458, -0.34645506, -0.34526143,  0.79088115,  1.64797129,
         0.11546285]])

## TensorFlow Regression

In [8]:
X_train.shape

(15788, 30)

In [10]:
model_regression = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_shape=(30,)),
        tf.keras.layers.Dense(32, activation="linear"),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(1, activation="linear"),
    ]
)

In [11]:
model_regression.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 64)                1984      
                                                                 
 dense_5 (Dense)             (None, 32)                2080      
                                                                 
 dropout_2 (Dropout)         (None, 32)                0         
                                                                 
 dense_6 (Dense)             (None, 16)                528       
                                                                 
 dropout_3 (Dropout)         (None, 16)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                 17        
                                                                 
Total params: 4,609
Trainable params: 4,609
Non-traina

In [12]:
model_regression.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss="mse")

In [13]:
model_regression.fit(X_train, y_train, epochs=50)

Epoch 1/50
494/494 [==============================] - 3s 3ms/step - loss: 10951.5664
Epoch 2/50
494/494 [==============================] - 2s 3ms/step - loss: 10375.4268
Epoch 3/50
494/494 [==============================] - 1s 3ms/step - loss: 10245.6445
Epoch 4/50
494/494 [==============================] - 2s 5ms/step - loss: 10202.4961
Epoch 5/50
494/494 [==============================] - 2s 5ms/step - loss: 10156.9141
Epoch 6/50
494/494 [==============================] - 2s 4ms/step - loss: 10101.6250
Epoch 7/50
494/494 [==============================] - 2s 4ms/step - loss: 10096.2168
Epoch 8/50
494/494 [==============================] - 1s 2ms/step - loss: 10076.5908
Epoch 9/50
494/494 [==============================] - 1s 2ms/step - loss: 10070.6855
Epoch 10/50
494/494 [==============================] - 1s 3ms/step - loss: 10023.9658
Epoch 11/50
494/494 [==============================] - 2s 5ms/step - loss: 9928.1318
Epoch 12/50
494/494 [==============================] - 2s 4ms/st

In [14]:
print(mean_absolute_error(y_test, model_regression.predict(X_test)))
print(mean_squared_error(y_test, model_regression.predict(X_test)))

124/124 [==============================] - 0s 3ms/step
48.095416940554266
124/124 [==============================] - 0s 2ms/step
9110.92049545914


## TensorFlow Classification

In [10]:
X_classification_train.shape

(14144, 6)

In [11]:
model_classification_1 = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_shape=(6,)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)

NameError: name 'tf' is not defined

In [88]:
model_classification_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="binary_crossentropy")

In [89]:
model_classification_1.fit(X_classification_train, y_classification_train, epochs=25)

Epoch 1/25
442/442 [==============================] - 1s 1ms/step - loss: 0.3519
Epoch 2/25
442/442 [==============================] - 0s 1ms/step - loss: 0.3182
Epoch 3/25
442/442 [==============================] - 0s 1ms/step - loss: 0.3131
Epoch 4/25
442/442 [==============================] - 1s 1ms/step - loss: 0.3089
Epoch 5/25
442/442 [==============================] - 0s 1ms/step - loss: 0.3050
Epoch 6/25
442/442 [==============================] - 0s 956us/step - loss: 0.3042
Epoch 7/25
442/442 [==============================] - 0s 930us/step - loss: 0.3009
Epoch 8/25
442/442 [==============================] - 0s 933us/step - loss: 0.3001
Epoch 9/25
442/442 [==============================] - 0s 941us/step - loss: 0.2984
Epoch 10/25
442/442 [==============================] - 0s 953us/step - loss: 0.2982
Epoch 11/25
442/442 [==============================] - 0s 939us/step - loss: 0.2984
Epoch 12/25
442/442 [==============================] - 0s 942us/step - loss: 0.2966
Epoch 13/25

In [90]:
pred_classification = np.around(model_classification_1.predict(X_classification_test, verbose=None))
print(classification_report(y_classification_test, pred_classification))
print(confusion_matrix(y_classification_test, pred_classification))

              precision    recall  f1-score   support

           0       0.98      0.77      0.86      1768
           1       0.81      0.98      0.89      1768

    accuracy                           0.88      3536
   macro avg       0.89      0.88      0.87      3536
weighted avg       0.89      0.88      0.87      3536

[[1358  410]
 [  29 1739]]


In [91]:
model_classification_1.save('../models/TensorFlow Classification2')

INFO:tensorflow:Assets written to: ../models/TensorFlow Classification2\assets


INFO:tensorflow:Assets written to: ../models/TensorFlow Classification2\assets


In [92]:
from tensorflow import keras
model = keras.models.load_model('../models/TensorFlow Classification2')

In [93]:
pred_classification = np.around(model.predict(X_classification_test, verbose=None))
print(classification_report(y_classification_test, pred_classification))
print(confusion_matrix(y_classification_test, pred_classification))

              precision    recall  f1-score   support

           0       0.98      0.77      0.86      1768
           1       0.81      0.98      0.89      1768

    accuracy                           0.88      3536
   macro avg       0.89      0.88      0.87      3536
weighted avg       0.89      0.88      0.87      3536

[[1358  410]
 [  29 1739]]


In [71]:
underSampler = RandomUnderSampler(random_state=42)
X_classification, y_classification = underSampler.fit_resample(X_classification,y_classification)

In [72]:
model_classification_2 = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_shape=(6,)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)

In [73]:
X_classification_train, X_classification_test, y_classification_train, y_classification_test = train_test_split(X_classification,
                                                                                                                y_classification,
                                                                                                                stratify=y_classification,
                                                                                                                test_size=0.2)

In [74]:
model_classification_2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="binary_crossentropy")

In [75]:
model_classification_2.fit(X_classification_train, y_classification_train, epochs=25)

Epoch 1/25
442/442 [==============================] - 3s 5ms/step - loss: 24959.6094
Epoch 2/25
442/442 [==============================] - 2s 5ms/step - loss: 3611.5322
Epoch 3/25
442/442 [==============================] - 1s 3ms/step - loss: 1087.2595
Epoch 4/25
442/442 [==============================] - 1s 3ms/step - loss: 529.2101
Epoch 5/25
442/442 [==============================] - 1s 3ms/step - loss: 314.8487
Epoch 6/25
442/442 [==============================] - 1s 3ms/step - loss: 244.9636
Epoch 7/25
442/442 [==============================] - 1s 3ms/step - loss: 103.8953
Epoch 8/25
442/442 [==============================] - 1s 3ms/step - loss: 98.8535
Epoch 9/25
442/442 [==============================] - 1s 2ms/step - loss: 118.5045
Epoch 10/25
442/442 [==============================] - 1s 3ms/step - loss: 2.0755
Epoch 11/25
442/442 [==============================] - 1s 3ms/step - loss: 1.1615
Epoch 12/25
442/442 [==============================] - 1s 3ms/step - loss: 0.9716
Epoc

In [76]:
pred_classification = np.around(model_classification_1.predict(X_classification_test, verbose=None))
print(classification_report(y_classification_test, pred_classification))
print(confusion_matrix(y_classification_test, pred_classification))

              precision    recall  f1-score   support

           0       0.50      0.98      0.66      1768
           1       0.03      0.00      0.00      1768

    accuracy                           0.49      3536
   macro avg       0.26      0.49      0.33      3536
weighted avg       0.26      0.49      0.33      3536

[[1734   34]
 [1767    1]]


## perceptron

In [30]:
X_new = np.hstack([np.ones((3, 1)), [[1,2],[3,4],[5,6]]])
X_new

array([[1., 1., 2.],
       [1., 3., 4.],
       [1., 5., 6.]])

In [33]:
def iterate_batches(inputs, outputs, batchsize):
        l = len(inputs)
        for ndx in range(0, l, batchsize):
            yield inputs[ndx:min(ndx + batchsize, l)], outputs[ndx:min(ndx + batchsize, l)]

class DenseLayer:
    def __init__(self, n_units, input_size=None, activation=None):
        
        self.n_units = n_units                                    
        self.input_size = input_size                               
        self.W = None                                                                                      
        self.A = None                              
        self.fn, self.df = self.select_activation_fn(activation)

    def select_activation_fn(self, activation):
        if activation == 'relu':
            fn = lambda x: np.maximum(0, x)
            df = lambda x: np.where(x < 0, 0.0, 1.0)
        elif activation == 'sigmoid':
            fn = lambda x: 1 / (1 + np.exp(-x))
            df = lambda x: fn(x) * fn(1 - x)
        elif activation == 'tanh':
            fn = lambda x: (np.exp(x) - np.exp(-1)) / (np.exp(x) + np.exp(-x))
            df = lambda x: 1 / np.cos(x) ** 2
        elif activation == 'linear':
            fn = lambda x: x
            df = lambda x: 1.0
        return fn, df

    def init_weights(self):
        self.W = np.random.randn(self.n_units, self.input_size + 1)
    
    def get_activation_value(self, X):
        m_examples = X.shape[0]
        X_new = np.hstack([np.ones((m_examples, 1)), X])
        Z = np.dot(X_new, self.W.T)
        A = self.fn(Z)
        self.A = A
        return A
    
    def backprop(self, delta, a):
        da = self.df(a)    
        return np.dot(delta, self.W)[:, 1:] * da

class SequentialModel:
    def __init__(self, layers):     
        input_size = layers[0].n_units       
        layers[0].init_weights()             
        for layer in layers[1:]:             
            layer.input_size = input_size
            input_size = layer.n_units
            layer.init_weights()
        self.layers = layers
        self.lr = None

    def forward(self, X):            
        out = self.layers[0].get_activation_value(X)
        for layer in self.layers[1:]:
            out = layer.get_activation_value(out)
        return out
    
    def add_dim(self, x):
        return np.hstack([np.ones((x.shape[0], 1)), x])

    def backward(self, X, y_pred, y_true):
        n_layers = len(self.layers)
        delta = y_pred - y_true
        a = y_pred
        dWs = {}
        for i in range(-1, -len(self.layers), -1):
            a = self.layers[i - 1].A       
            dWs[i] = np.dot(delta.T, self.add_dim(a))
            delta = self.layers[i].backprop(delta, a)

        dWs[-n_layers] = np.dot(delta.T, self.add_dim(X))

        for k, dW in dWs.items():
            self.layers[k].W -= self.lr * dW
            
    def fit(self, X, y, epochs, batchsize=32, lr=0.01):
        
        self.lr = lr

        for i in range(epochs):

            for batch in iterate_batches(X, y, batchsize=batchsize):
                
                X_batch = batch[0]
                y_batch = batch[1]
                if not isinstance(y_batch, np.ndarray):
                    y_batch = y_batch.to_numpy()
                y_batch = y_batch.reshape(-1, 1)
                y_pred = model.forward(X_batch)
                model.backward(X_batch, y_pred, y_batch)
                

model = SequentialModel([
DenseLayer(6, activation='sigmoid', input_size=X_train2.shape[1]),
DenseLayer(4, activation='tanh'),
DenseLayer(3, activation='relu'),
DenseLayer(1, activation='sigmoid')
])

model.fit(X_train2, y_train2, epochs = 5, batchsize=32)
pred = model.forward(X_test2)
pred = np.where(pred > 0.5, 1, 0)
print(classification_report(y_test2, pred))


model = SequentialModel([
DenseLayer(6, activation='relu', input_size=X_train.shape[1]),
DenseLayer(4, activation='linear'),
DenseLayer(3, activation='relu'),
DenseLayer(1, activation='linear')
])

model.fit(X_train, y_train, epochs = 25, batchsize=32)
pred = model.forward(X_test)
print(f'MAE: {mean_absolute_error(y_test, pred)}')
print(f'MSE: {mean_squared_error(y_test, pred)}')


C:\Users\user\AppData\Local\Temp\ipykernel_17548\1816284130.py:20: RuntimeWarning: overflow encountered in exp
  fn = lambda x: 1 / (1 + np.exp(-x))


              precision    recall  f1-score   support

           0       0.77      0.32      0.45      1768
           1       0.57      0.90      0.70      1768

    accuracy                           0.61      3536
   macro avg       0.67      0.61      0.57      3536
weighted avg       0.67      0.61      0.57      3536



C:\Users\user\AppData\Local\Temp\ipykernel_17548\1816284130.py:43: RuntimeWarning: invalid value encountered in multiply
  return np.dot(delta, self.W)[:, 1:] * da


MAE: 95.52571573346846
MSE: 19514.51735495313
